TP - Introduction à la programmation orientée objet (suite)
=====

# Objectif

Dans ce deuxième TP sur la programmation orientée objet, nous allons simuler le fonctionnement (extrêmement simplifié) d'une centrale nucléaire en utilisant le paradigme objet en Python. Vous allez dans un premier temps modéliser différents composants de la centrale, tels que le(s) réacteur(s), le(s) système(s) de refroidissement et les capteurs. Ensuite, vous intégrerez ces composants afin de simuler le fonctionnement global de la centrale. Enfin vous réaliserez quelques tests sur votre centrale.

# Partie 1 : modélisation des composants de la centrale

Définissez chacune des classes suivantes :

## 1.1. Classe `Reactor`

Cette classe représente un réacteur nucléaire. Elle doit avoir les attributs suivants :

- `power` : la puissance fournie par le réacteur (pourcentage de sa puissance maximale).
- `temperature` : la température du réacteur.

On trouve aussi les constantes :

- `MAX_OPERATING_POWER` fixée à 80 % 
- `NORMAL_OPERATING_POWER` fixée à 50 % 
- `MAX_ALLOWABLE_TEMPERATURE` fixée à 345°C

Le constructeur :

- `__init__(self)` : crée un réacteur dont la puissance fournie est à 0 % de sa puissance maximale et dont la température est à 0°C.

Et les méthodes :

- `increase_power(self, percentage)` : augmente la puissance du réacteur de `percentage` % (la valeur ne pouvant augmenter au-delà de  `MAX_OPERATING_POWER`). Cette augmentation a pour effet d'induire une augmentation de la température du réacteur de `percentage`*5°C (température qui sera verrouillée à `MAX_ALLOWABLE_TEMPERATURE`).
- `decrease_power(self, percentage)` : diminue la puissance du réacteur de `percentage` % (la valeur ne pouvant diminuer en-dessous de 0 %). Cette diminution a pour effet d'induire une diminution de la température du réacteur de 5°C. Évidemment, puissance et température ne pourront être négatifs.
- `cool_down(self)` : abaisse la température du réacteur de 10°C (cet abaissement pourra avoir lieu lorsque le système de refroidsissement sera activé).
- `get_power(self)` : retourne la puissance actuelle du réacteur.
- `get_temperature(self)` : retourne la température actuelle du réacteur.

In [58]:
#
# Reactor class to define
#

## 1.2. Classe `CoolingSystem`

Cette classe représente le système de refroidissement de la centrale. Elle doit avoir les attributs suivants :

- `status` : état actuel du système (`active` ou `inactive`).
- `coolant_temperature` : température du liquide de refroidissement.

Le constructeur :

- `__init__(self)` : crée un système de refroidissement dont l'état est inactif et dont la température du liquide est à 25°C.

Et les méthodes :

- `activate(self)` : active le système de refroidissement (l'activation augmentant sa température de +5°C).
- `deactivate(self)` : désactive le système de refroidissement (la désactivation diminuant sa température de +5°C).
- `get_coolant_temperature(self)` : retourne la température du liquide de refroidissement.

In [19]:
#
# CoolingSystem class to define
#

## 1.3. Classe `Sensor`

Cette classe représente un capteur générique. Elle doit avoir les attributs suivants :

- `sensor_id` : identifiant du capteur.
- `location` : emplacement du capteur dans la centrale.
- `data` : données lues par le capteur (initialement on lui affecte l'objet `None`).
- `error_margin` : erreur de mesure du capteur.

Le constructeur : 

- `__init__(self, sensor_id, location, error_margin=0.05)` : crée un capteur ayant un identifiant (chaîne de caractères), une localisation (chaîne de caractères), et une erreur de lecture spécifique.

Et les méthodes :

- `read_data(self, source)` : lit les données depuis la source considérée. Cette méthode doit impérativement être redéfinie dans les classes filles.
- `display_data(self)` : affiche les informations du capteur (identifiant, position et données lues).
- `add_measurement_error(self, value)`: ajoute à value une erreur de lecture aléatoire pour ce capteur (comprise entre -`error_margin` et +`error_margin`) et retourne la nouvelle valeur. 

In [20]:
import random
import time

#
# Sensor class to complete
#
class Sensor:
    def __init__(self, sensor_id, location, error_margin=0.05):
        self.sensor_id = sensor_id
        self.location = location
        self.data = None
        self.error_margin = error_margin

    def read_data(self, source):
        """Reads data from a given source."""
        raise NotImplementedError("La méthode read_data() doit être implémentée par les classes filles.")

    def display_data(self):
        """Displays data."""
        if self.data is not None:
            print(f"Capteur : {self.sensor_id}, Emplacement : {self.location}, Données : {self.data}")
        else:
            print("Aucune donnée disponible.")


## 1.4. Classe `TemperatureSensor`

Cette classe représente un capteur de température. Elle hérite de la classe `Sensor` et doit avoir les attributs suivants :

- `unit` : unité de mesure du capteur (dans notre cas °C).

Le constructeur : 

- `__init__(self, sensor_id, location, unit="Celsius")` : crée un capteur de températur ayant un identifiant (chaîne de caractères), une localisation (chaîne de caractères), et une unité pour la mesure.

Et les méthodes :

- `read_data(self, reactor)` : lit la température du réacteur (n'oubliez pas de prendre en compte son erreur de mesure).
- `display_data(self)` : affiche les informations du capteur (identifiant, position et température lue).


In [56]:
#
# TemperatureSensor to define
#

# Partie 2 : intégration et simulation

## 2.1. Intégration des composants

Créez une classe `NuclearPlant` qui intègre les composants précédents. Cette classe doit avoir les attributs suivants :

- `reactor` : une instance de la classe `Reactor`.
- `cooling_system` : une instance de la classe `CoolingSystem`.
- `sensors` : la liste des capteurs (ici 2 instances de la classe `TemperatureSensor`).
- `min_threshold_temperature` : le seuil de température permettant la désactivation du système de refroidissement.
- `max_threshold_temperature` : le seuil de température permettant l'activation du système de refroidissement.

Le constructeur : 

- `__init__(self, min_threshold_temperature=320, max_threshold_temperature=450)` : crée une centrale nucléaire composée d'un reacteur, d'un système de refroidissement, d'une liste de capteurs (dans notre cas 2 capteurs de température), et ayant 2 températures de bascule permettant de désactiver ou d'activer automatiquement le système de refroidissement.

Et la méthode :

- `simulate(self)` : simule le fonctionnement de la centrale pendant un certain temps (par exemple, 10 minutes). Pendant cette simulation, la température du réacteur et du liquide de refroidissement doit être mise à jour régulièrement. Si la température du réacteur dépasse
  `max_threshold_temperature` alors le système de refroidissement doit être activé, et lors d'un refroidissement, on choisira de désactiver
  celui-ci dès que la température descend sous le seuil `min_threshold_temperature`.


In [59]:
#
# NuclearPlant class to define
#

## 2.2. Interface graphique

Nous utilisons un certain nombre de *widgets* Jupyter pour créer une interface graphique permettant de contrôler le réacteur de la centrale ainsi que le système de refroidissement. Si vos classes sont correctement définies, vous devriez pouvoir contrôler la puissance du réacteur, activer/désactiver le refroidissement et visualiser la température actuelle du réacteur.
La documentation sur les *widgets* se trouve à cette adresse : [https://ipywidgets.readthedocs.io/en/stable/](https://ipywidgets.readthedocs.io/en/stable/)

In [60]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Initialisation de l'objet NuclearPlant
plant = NuclearPlant()

# Interface graphique
power_slider = widgets.IntSlider(value=plant.reactor.NORMAL_OPERATING_POWER, min=0, max=100, description="Puissance (%)", continuous_update=False)
cooling_button = widgets.ToggleButton(value=False, description="Refroidissement", icon="check", style=dict(
    font_weight='bold',
    font_variant="small-caps",
    text_color='blue',
    text_decoration='underline'
))
reactor_temperature_progress = widgets.FloatProgress(value=300, min=0, max=1000, description="Temp. Réacteur :", bar_style="info")
cooling_status_label = widgets.Label(value="État du système de refroidissement : {}".format(plant.cooling_system.status))
sensor_output = widgets.Output()
output = widgets.Output()  

def update_display():
    with output:
        clear_output(wait=True)

        # Read data from sensors
        for sensor in plant.sensors:
            sensor.read_data(plant.reactor)
        
        # Get all data 
        temps = [sensor.data for sensor in plant.sensors]
        
        # ---------------------------------------------------------------------------------
        # 1) Update the progress bar color according to the average temperature
        # (do not consider None -- that is the case when starting).
        # 2) 
        # IF mean temperature < min_threshold_temperature in plant
        # THEN reactor_temperature_progress.bar_style = "success" (green color)
        # ELSE IF min_threshold_temperature in plant < mean temperature
        # THEN reactor_temperature_progress.bar_style = "warning" (orange color)
        # ELSE reactor_temperature_progress.bar_style = "danger" (red color)
        # TO COMPLETE
        # ---------------------------------------------------------------------------------
        
        reactor_temperature_progress.value = avg_temp
        cooling_status_label.value = "État du système de refroidissement : {}".format(plant.cooling_system.status)
        display(reactor_temperature_progress, cooling_status_label)
        
        # Display all data
        with sensor_output:
            clear_output(wait=True)  
            for sensor in plant.sensors:
                sensor.display_data()
        display(sensor_output)

def on_power_change(change):
    if change["new"] > change["old"]:
        plant.reactor.increase_power(change["new"] - change["old"])
    else:
        plant.reactor.decrease_power(change["old"] - change["new"])
    update_display()

def on_cooling_button_click(change):
    if change["new"]:
        plant.cooling_system.activate()
    else:
        plant.cooling_system.deactivate()
    update_display()

power_slider.observe(on_power_change, names="value")
cooling_button.observe(on_cooling_button_click, names="value")

display(widgets.VBox([widgets.Label(value="Contrôle de la centrale nucléaire :"), power_slider, cooling_button, output]))
update_display()

# Partie 3 : scénarios de test

Vous allez tester votre centrale en créant 3 différents scénarios.

## 3.1. Que se passe-t-il si le réacteur surchauffe ?

Définissez la fonction `test_reactor_overheat(NuclearPlant)` qui réalisera les opérations suivantes :

1. Augmentation de la puissance du réacteur à un niveau élevé (par exemple 90%).
2. Récupération et affichage de la moyenne des températures lues par les deux capteurs pendant une dizaine de minutes (même avec l'erreur de mesure, les deux devraient indiquer une température élevée).
3. Test de l'activation automatique du système de refroidissement lorsque la température moyenne dépasse le seuil.


In [47]:
#
# Function test_reactor_overheat(NuclearPlant) to define
#

## 3.2. Qu'elle est la réaction si le système de refroidissement est désactivé pendant une surchauffe ?

Définissez la fonction `test_cooling_system_deactivated(NuclearPlant)` qui réalisera les opérations suivantes :

1. Augmentation de la puissance du réacteur à un niveau élevé (par exemple, 90%).
2. Désactivation du système de refroidissement.
3. Récupération et affichage de la moyenne des températures lues par les deux capteurs pendant une dizaine de minutes.
4. Vérification de l'activation automatique du système de refroidissement une fois que la température moyenne dépasse un certain seuil critique.


In [48]:
#
# Function test_cooling_system_deactivation(NuclearPlant) to define
#


## 3.3. Quelle est la température maximale que le réacteur peut atteindre avant que le système de refroidissement ne soit activé ?

Définissez la fonction `test_max_temperature_before_cooling(NuclearPlant)` qui réalisera les opérations suivantes :

1. Augmentation progressive de la puissance du réacteur par paliers de 10% tout en observant la moyenne des lectures des capteurs.
2. Affichage de la température moyenne au moment où le système de refroidissement s'active automatiquement.

In [49]:
#
# Function test_max_temperature_before_cooling(NuclearPlant) to define
#


### Exécution des 3 tests

In [50]:
plant = NuclearPlant()

test_reactor_overheat(plant)
test_cooling_system_deactivation(plant)
test_max_temperature_before_cooling(plant)

Test de surchauffe du réacteur
Température moyenne : 334.38°C
Température moyenne : 336.04°C
Température moyenne : 369.17°C
Température moyenne : 397.24°C
Température moyenne : 402.67°C
Température moyenne : 432.73°C
Température moyenne : 439.26°C
Température moyenne : 477.93°C
Température moyenne : 488.59°C
Température moyenne : 542.57°C
La température du réacteur a dépassé la limite sécuritaire.
Le système de refroidissement devrait être activé !

Test de désactivation du système de refroidissement pendant une surchauffe
Température moyenne : 545.78°C
Température moyenne : 560.78°C
Température moyenne : 603.19°C
Température moyenne : 611.14°C
Température moyenne : 639.35°C
Température moyenne : 680.91°C
Température moyenne : 683.41°C
Température moyenne : 676.14°C
Température moyenne : 710.76°C
Température moyenne : 727.49°C
La température du réacteur est restée dans la plage sécuritaire.

Récupération de la température maximale avant activation du système de refroidissement
Températ

### Interface graphique pour la simulation des scénarios


In [63]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Widgets to control our nuclear plant
power_slider = widgets.IntSlider(value=50, min=0, max=100, description="Puissance (%)", continuous_update=False)
cooling_button = widgets.ToggleButton(value=False, description="Refroidissement", icon="check")
reactor_temperature_progress = widgets.FloatProgress(value=300, min=0, max=1000, description="Temp. moyenne :", bar_style="info")
sensor1_label = widgets.Label(value="Capteur 1 : 300°C")
sensor2_label = widgets.Label(value="Capteur 2 : 300°C")

# Buttons for our 3 tests
test1_button = widgets.Button(description="Test 1 : surchauffe du réacteur")
test2_button = widgets.Button(description="Test 2 : désactivation du refroidissement")
test3_button = widgets.Button(description="Test 3 : température maximale avant refroidissement")

output = widgets.Output()

def update_display():
    with output:
        clear_output(wait=True)
        
        # Read data from sensors
        for sensor in plant.sensors:
            sensor.read_data(plant.reactor)
        
        # Compute the average temperature from sensor data
        avg_temp = sum([sensor.data for sensor in plant.sensors]) / len(plant.sensors)
        
        # Update the progress bar color according to the average temparature
        if avg_temp < plant.min_threshold_temperature:
            reactor_temperature_progress.bar_style = "success"  # Vert
        elif plant.min_threshold_temperature <= avg_temp < plant.max_threshold_temperature:
            reactor_temperature_progress.bar_style = "warning"  # Jaune
        else:
            reactor_temperature_progress.bar_style = "danger"   # Rouge
        
        reactor_temperature_progress.value = avg_temp
        sensor1_label.value = f"Capteur 1 : {plant.sensors[0].data:.2f}°C"
        sensor2_label.value = f"Capteur 2 : {plant.sensors[1].data:.2f}°C"
        
        display(reactor_temperature_progress, sensor1_label, sensor2_label)

def on_power_change(change):
    if change["new"] > change["old"]:
        plant.reactor.increase_power(change["new"] - change["old"])
    else:
        plant.reactor.decrease_power(change["old"] - change["new"])
    update_display()

def on_cooling_button_click(change):
    if change["new"]:
        plant.cooling_system.activate()
    else:
        plant.cooling_system.deactivate()
    update_display()

def on_test1_button_click(b):
    test_reactor_overheat(plant)
    update_display()

def on_test2_button_click(b):
    test_cooling_system_deactivation(plant)
    update_display()

def on_test3_button_click(b):
    test_max_temperature_before_cooling(plant)
    update_display()

power_slider.observe(on_power_change, names="value")
cooling_button.observe(on_cooling_button_click, names="value")
test1_button.on_click(on_test1_button_click)
test2_button.on_click(on_test2_button_click)
test3_button.on_click(on_test3_button_click)

plant = NuclearPlant()

display(widgets.VBox([widgets.Label(value="Test de la centrale nucléaire :"), power_slider, cooling_button, output, test1_button, test2_button, test3_button]))
update_display()
